# Network Plug and Play
This workbook contains examples of the network plug and play API for APIC-EM using the uniq library.

First get a connection to the __controller__

In [44]:
from login import login
import json

apic = login()

## Get a list of projects

getPnpSiteByRange


In [45]:
# use this to save the name of a project with devices in it
projectName = ''

# get a list of all projects
projects = apic.pnpproject.getPnpSiteByRange()

# now print them out
for project in projects.response:
    
    # save the name of a project with devicerules
    if project.deviceCount > 0:
        projectName = project.siteName
    
    # formatted print
    print("Project {project}: Rule count {count}".format(
                            project=project.siteName, 
                            count=project.deviceCount))

Project 4331 Primary: Rule count 0
Project Branch: Rule count 1
Project Branch Brno 01: Rule count 0
Project Branch Ostrava 01: Rule count 0
Project Branch Ostrava 02: Rule count 0
Project Branch Plzen 01: Rule count 0
Project Branch Plzen 02: Rule count 0
Project Branch Plzen 03: Rule count 0
Project Branch3-Site: Rule count 1
Project Canberra-3114: Rule count 0
Project Canberra-3882: Rule count 0
Project Central branch 1: Rule count 1
Project Melbourne-4829: Rule count 1
Project Melbourne-6230: Rule count 1
Project New Building: Rule count 2
Project Sydney-4829: Rule count 2
Project Sydney-6230: Rule count 2
Project TEST123: Rule count 2
Project Test_new: Rule count 3
Project WS-LND-3114: Rule count 1
Project WS-LND-4829: Rule count 0
Project WS-Perth-3114: Rule count 2
Project WS-Perth-4829: Rule count 2
Project blueCloud: Rule count 4


Look up the project by name, and then get all of the device rules for the site

In [46]:
# look up project by name
project = apic.pnpproject.getPnpSiteByRange(siteName=projectName)

if project.response == []:
    print("Project '%s' not found." % projectName)
    raise ValueError("invalid project name")

# there will be only one match.  Cannot have multiple projects with the same name
print('Project:{project}, device count{count}\n'.format(project=projectName,
                                                              count=project.response[0].deviceCount))
# get the device rules
devices = apic.pnpproject.getPnpSiteDevicesBySiteNameAndRange(projectId=project.response[0].id)
for device in devices.response:
    print("Device {hostName:10s}: serialNumber {serialNumber:11s}: State {state:14s}\n".format(
                            hostName=device.hostName,
                            serialNumber= device.serialNumber,
                            state=device.state
                            ))


Project:blueCloud, device count4

Device rfra101   : serialNumber 12345678978: State PENDING       

Device rmuc101   : serialNumber 12345678909: State PENDING       

Device rmuc102   : serialNumber 12345678908: State PENDING       

Device rfra102   : serialNumber 12345678907: State PENDING       



***
Generate a random suffix.  This is required as projectnames, serial numbers and filenames need to be unique across the controller.  If you are using this on a dedicated controller, no need for this.

In [47]:
import random
randSuffix= ('{num:03d}'.format(num=int(random.random() * 999)))
print("RandomSuffix",randSuffix)

RandomSuffix 552


### File Upload
Upload a dummy configuration file.  

In [48]:
filename = "/tmp/CONFIG"+randSuffix
with open(filename, "w") as filehandle:
    print("hostname fred", file=filehandle)
file_result = apic.file.uploadFile(nameSpace="config", fileUpload=filename)
file_id = file_result.response.id
print("Configuration File_id:", file_id)


Configuration File_id: e5b5c11f-927f-465c-8079-accfcbd1843f


### Creating project

In [49]:
project_name = "ProjectPnP" + randSuffix

print ("creating project:{project}".format(project=project_name))
pnp_task_response= apic.pnpproject.createPnpSite(project=[{'siteName' :project_name}])
task_response = apic.task_util.wait_for_task_complete(pnp_task_response, timeout=5)

# 'progress': '{"message":"Success creating new site","siteId":"6e059831-b399-4667-b96d-8b184b6bc8ae"}'
progress = task_response.progress
project_id = json.loads(progress)['siteId']
print ("ProjectId:%s" % project_id)

creating project:ProjectPnP552
ProjectId:2fa1c860-76c0-4b0d-8246-c80c96cc6d83


### Creating a rule

In [50]:
rule_data = [{
        "serialNumber": "12345678" + randSuffix,
        "platformId":  "3850",
        "hostName": "test_host",
        "configId" : file_id,
        "pkiEnabled": True
}]

    
rule_task = apic.pnpproject.createPnpSiteDevice(projectId=project_id, rule=rule_data)
task_response = apic.task_util.wait_for_task_complete(rule_task, timeout=5)
progress = task_response.progress
print("Rule Status:", progress)

Rule Status: {"message":"Success creating new site device(rule)","ruleId":"8d347bb9-d996-4425-ab73-a00a2d37e013"}


### Project deletion
This deletes the project (by project_id).  All rules and device information is removed.

In [51]:
pnp_task_response = apic.pnpproject.deletePnpSiteByID(projectId=project_id, deleteRule=1, deleteDevice=1)
task_response = apic.task_util.wait_for_task_complete(pnp_task_response, timeout=5)
print (apic.serialize(task_response))

{'id': 'a70d461b-2e3f-4177-81f4-0f6176b1cc05', 'isError': False, 'endTime': 1498434485937, 'startTime': 1498434485911, 'version': 1498434485911, 'progress': 'Success Deleting ZTD Site: id# 2fa1c860-76c0-4b0d-8246-c80c96cc6d83', 'serviceType': 'Ztd Service', 'rootId': 'a70d461b-2e3f-4177-81f4-0f6176b1cc05'}


### File Deletion
This is one of the few operations that are synchronous (no waiting)

In [52]:
file_task = apic.file.deleteFile(fileId=file_id)
print(apic.serialize(file_task))

{'version': '1.0', 'response': 'File id=e5b5c11f-927f-465c-8079-accfcbd1843f is deleted successfully'}
